In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt

In [ ]:
# Setting the drawing parameters for the academic paper style
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Arial', 'DejaVu Sans Mono', 'Times New Roman', 'DejaVu Serif'],
    'font.size': 12,
    'axes.labelsize': 14,
    'axes.titlesize': 16,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'legend.fontsize': 12,
    'figure.dpi': 300,
    'savefig.dpi': 300,
    'savefig.bbox': 'tight',
    'savefig.pad_inches': 0.1
})

In [ ]:
# Reading kinetic parameter data
file_path = 'kinodynamic_sorted.xlsx'
sheet_name = 'Sheet1'
data = pd.read_excel(file_path, sheet_name=sheet_name)
data = data.rename(columns={'Unnamed: 0': 'Category'})

# Read the success rate data for each algorithm
algorithms = {
    'EGO-Planner': 'Pre-treatment_EGO_planner_success_rate.xlsx',
    'Fast-Planner': 'Pre-treatment_Fast_Planner_success_rate.xlsx',
    'PGO': 'Pre-treatment_PGO_success_rate.xlsx',
    'NavRL': 'Pre-treatment_NavRL_success_rate.xlsx',
}

# Calculate the average success rate of each algorithm
algorithm_means = {}
for algo_name, file_path in algorithms.items():
    algo_data = pd.read_excel(file_path, sheet_name=sheet_name)
    algo_data = algo_data.iloc[0:, 1:]
    
    # Calculate the average value of each row
    means = np.zeros((algo_data.shape[0], 1))
    for i in range(algo_data.shape[0]):
        means[i, 0] = np.mean(algo_data.iloc[i, :])
    
    algorithm_means[algo_name] = means

kinodynamic_params = ['N', 'XY', 'Z']
param_labels = [r'$\mathrm{TWR}_{\max}$', r'$\alpha_{xy,\max}(rad/s^2)$', r'$\alpha_{z,\max}(rad/s^2)$']
file_names = ['TWR_vs_success_rate', 'alpha_xy_vs_success_rate', 'alpha_z_vs_success_rate']

colors = ["#3184CD", "#F4811C", "#E22F13", "#786C6C"]
markers = ['o', 'v', 's', '*']
marker_sizes = [80, 80, 80, 80]

for i, (param, param_label, file_name) in enumerate(zip(kinodynamic_params, param_labels, file_names)):
    fig, ax = plt.subplots(figsize=(6, 4))
    
    for j, (algo_name, means) in enumerate(algorithm_means.items()):
        ax.scatter(data[param], means,
                  c=colors[j], marker=markers[j], 
                  s=marker_sizes[j], label=algo_name, alpha=0.8)
    
    ax.set_xlabel(param_label)
    ax.set_ylabel('Success Rate')
    ax.grid(True, linestyle='--', alpha=0.6)
    
    ax.set_ylim(0, 1)
    
    # Use logarithmic scale for XY parameters
    if param == 'XY':
        ax.set_xscale('log')
    
    ax.legend(loc='upper right', frameon=True, fancybox=True, shadow=False, 
             fontsize=11, handletextpad=0.5, borderpad=0.5)
    
    plt.tight_layout()
    
    plt.savefig(f'{file_name}.png', dpi=600)
    plt.savefig(f'{file_name}.pdf', dpi=800)
    
    plt.close(fig)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, (ax, param, param_label) in enumerate(zip(axes, kinodynamic_params, param_labels)):
    for j, (algo_name, means) in enumerate(algorithm_means.items()):
        ax.scatter(data[param], means,
                  c=colors[j], marker=markers[j], 
                  s=marker_sizes[j], label=algo_name, alpha=0.8)
    
    ax.set_xlabel(param_label)
    ax.grid(True, linestyle='--', alpha=0.6)
    
    ax.set_ylim(0, 1)
    
    if param == 'N':
        ax.set_ylabel('Success Rate')
    
    if param == 'XY':
        ax.set_xscale('log')
    
    if i == 2:
        ax.legend(loc='upper right', frameon=True, fancybox=True, shadow=False, 
                 fontsize=11, handletextpad=0.5, borderpad=0.5)

plt.tight_layout()

plt.savefig('algorithm_success_rate_analysis_combined.png', dpi=600)
plt.savefig('algorithm_success_rate_analysis_combined.pdf', dpi=800)

plt.show()